In [ ]:
import time
from urllib import urlencode
import selenium.webdriver
from bs4 import BeautifulSoup
import itertools
import pandas as pd

def first_search_boardgamegeek(query, browser, delay=10):
    search_url = "https://boardgamegeek.com/browse/boardgame/page/{}".format(query)
    browser.get(search_url)
    #time.sleep(delay)  # Delay is unnecessary for this site
    return browser.page_source

def get_game_url(product_tag):
    title_elements = product_tag.select('a')
    if title_elements:
        product_title = title_elements[1].get('href')
        return product_title
    
def get_boardgamegeek_product_details(query, browser):
    html = first_search_boardgamegeek(query, browser)
    soup = BeautifulSoup(html, 'html.parser')
    product_tags = soup.find_all("tr", id="row_")
    product_tags.extend(soup.find_all("tr", id="row_"))
    return [get_game_url(t) for t in product_tags]

browser = selenium.webdriver.Firefox()
product_details = []
for i in xrange(1, 136):
    product_details.append(get_boardgamegeek_product_details(i, browser)[:100])
browser.quit()

product_details = list(itertools.chain.from_iterable(product_details))

def second_search_boardgamegeek(query, browser, delay=10):
    search_url = "https://boardgamegeek.com{}".format(query)
    browser.get(search_url)
    #time.sleep(delay)  # Wait a few seconds before getting the HTML source
    return browser.page_source

def get_boardgamegeek_game_details(query, browser):
    html = second_search_boardgamegeek(query, browser)
    soup = BeautifulSoup(html, 'html.parser')
    product_tags = soup.find_all("div", class_="content")
    return [(get_game_title(t), get_secondary_game_info(t), get_the_rest(t)) for t in product_tags]

def get_game_title(product_tag):
    title_elements = product_tag.find_all("div", class_="game-header-title-info")
    if title_elements:
        game_title = title_elements[1].find_all('h1')
        year_published = game_title[0].text.split()[-1:]
        game_title = ' '.join(game_title[0].text.split()[:-1])
        return game_title, year_published
    
def get_secondary_game_info(product_tag):
    title_elements = product_tag.find_all('li', attrs={'class':'gameplay-item'})
    if title_elements:
        game_info = title_elements
        num_players = game_info[0].text.split()
        min_players = num_players[0][0]
        max_players = num_players[0][2:]
        best_players = num_players[-1]
        play_time = game_info[1].text.split()[0].split(u'\u2013')
        age = game_info[2].text.split()[1].strip('+')
        complexity = game_info[3].text.split()[2]
        return min_players, max_players, best_players, play_time, age, complexity
    
def get_the_rest(product_tag):
    title_elements = product_tag.find_all('li', attrs={'class':'outline-item'})
    if title_elements:
        game_title = title_elements
        designers = game_title[3].find_all('a')[1:]
        for i, designer in enumerate(designers):
            designers[i] = designer.text
        artists = game_title[4].find_all('a')[1:]
        for i, artist in enumerate(artists):
            artists[i] = artist.text
        publishers = game_title[5].find_all('a')[1:]
        for i, publisher in enumerate(publishers):
            publishers[i] = publisher.text
        categories = game_title[6].find_all('a')[1:]
        for i, categorie in enumerate(categories):
            categories[i] = categorie.text
        mechanisms = game_title[7].find_all('a')[1:]
        for i, mechanism in enumerate(mechanisms):
            mechanisms[i] = mechanism.text
        return designers, artists, publishers, categories, mechanisms
    
browser = selenium.webdriver.Firefox()
game_details = []
for i in product_details:
    new_search = '{}/credits'.format(i)
    game_info = get_boardgamegeek_game_details(new_search, browser)[0]
    all_info = [game_info[0][0], game_info[0][1][0], game_info[1][0], \
                    game_info[1][1], game_info[1][2], game_info[1][3], game_info[1][4], game_info[1][5], \
                   game_info[2][0], game_info[2][1], game_info[2][2], game_info[2][3], game_info[2][4]]
    if all_info[3] == '':
        all_info[3] = all_info[2]
    try:
        all_info[5][1]
    except IndexError:
        all_info[5].append(all_info[5][0])
    game_details.append(all_info)
browser.quit()

board_game_data = pd.DataFrame\
(game_details, index=range(1, 13501), columns=[ \
                         'Title', 'Year Published',\
                         'Min Players', 'Max Players', 'Best Players',\
                         'Play Time', 'Age', 'Complexity', 'Designers',\
                         'Artists', 'Publishers', 'Categories',\
                         'Mechanisms'])

board_game_data.to_csv('boardgame_data.csv', encoding='utf-8')

#GAME CALLED ¥€$ SHOULD SEARCH https://boardgamegeek.com////credits, NOT WHATEVER IS IN product_details[13107]. NEED TO FIX THIS LATER.
#IMPORTED ¥€$ DATA MANUALLY

In [ ]:
def second_search_boardgamegeek(query, browser, delay=10):
    search_url = "https://boardgamegeek.com{}".format(query)
    browser.get(search_url)
    #time.sleep(delay)  # Wait a few seconds before getting the HTML source
    return browser.page_source

def get_boardgamegeek_game_details(query, browser):
    html = second_search_boardgamegeek(query, browser)
    soup = BeautifulSoup(html, 'html.parser')
    product_tags = soup.find_all("div", class_="content")
    return [(get_game_title(t), get_secondary_game_info(t), get_the_rest(t)) for t in product_tags]

def get_game_title(product_tag):
    title_elements = product_tag.find_all("div", class_="game-header-title-info")
    if title_elements:
        game_title = title_elements[1].find_all('h1')
        year_published = game_title[0].text.split()[-1:]
        game_title = ' '.join(game_title[0].text.split()[:-1])
        return game_title, year_published
    
def get_secondary_game_info(product_tag):
    title_elements = product_tag.find_all('li', attrs={'class':'gameplay-item'})
    if title_elements:
        game_info = title_elements
        num_players = game_info[0].text.split()
        min_players = num_players[0][0]
        max_players = num_players[0][2:]
        best_players = num_players[-1]
        play_time = game_info[1].text.split()[0].split(u'\u2013')
        age = game_info[2].text.split()[1].strip('+')
        complexity = game_info[3].text.split()[2]
        return min_players, max_players, best_players, play_time, age, complexity
    
def get_the_rest(product_tag):
    title_elements = product_tag.find_all('li', attrs={'class':'outline-item'})
    if title_elements:
        game_title = title_elements
        designers = game_title[3].find_all('a')[1:]
        for i, designer in enumerate(designers):
            designers[i] = designer.text
        artists = game_title[4].find_all('a')[1:]
        for i, artist in enumerate(artists):
            artists[i] = artist.text
        publishers = game_title[5].find_all('a')[1:]
        for i, publisher in enumerate(publishers):
            publishers[i] = publisher.text
        categories = game_title[6].find_all('a')[1:]
        for i, categorie in enumerate(categories):
            categories[i] = categorie.text
        mechanisms = game_title[7].find_all('a')[1:]
        for i, mechanism in enumerate(mechanisms):
            mechanisms[i] = mechanism.text
        return designers, artists, publishers, categories, mechanisms
    
browser = selenium.webdriver.Firefox()
game_details = []
for i in product_details:
    new_search = '{}/credits'.format(i)
    game_info = get_boardgamegeek_game_details(new_search, browser)[0]
    all_info = [game_info[0][0], game_info[0][1][0], game_info[1][0], \
                    game_info[1][1], game_info[1][2], game_info[1][3], game_info[1][4], game_info[1][5], \
                   game_info[2][0], game_info[2][1], game_info[2][2], game_info[2][3], game_info[2][4]]
    if all_info[3] == '':
        all_info[3] = all_info[2]
    try:
        all_info[5][1]
    except IndexError:
        all_info[5].append(all_info[5][0])
    game_details.append(all_info)
browser.quit()

board_game_data = pd.DataFrame\
(game_details, index=range(1, 13501), columns=[ \
                         'Title', 'Year Published',\
                         'Min Players', 'Max Players', 'Best Players',\
                         'Play Time', 'Age', 'Complexity', 'Designers',\
                         'Artists', 'Publishers', 'Categories',\
                         'Mechanisms'])

board_game_data.to_csv('boardgame_data.csv', encoding='utf-8')

#GAME CALLED ¥€$ SHOULD SEARCH https://boardgamegeek.com////credits, NOT WHATEVER IS IN product_details[13107]. NEED TO FIX THIS LATER.
#IMPORTED ¥€$ DATA MANUALLY

In [27]:
import pandas as pd
import numpy as np

In [194]:
bg_data = pd.read_csv('data/boardgame_data.csv')
bg_data = bg_data.drop('Unnamed: 0', axis=1)
ind = []
bg_data['Year Published'] = bg_data['Year Published'].apply(lambda x: x.strip('\(\)'))
for i in xrange(len(bg_data)):
    try:
        int(bg_data['Year Published'][i])
    except ValueError:
        ind.append(i)
for i in ind:
    try:
        bg_data.set_value(i, 'Title', bg_data['Title'][i]+' '+bg_data['Year Published'][i])
        bg_data.set_value(i, 'Year Published', 0)
    except TypeError:
        bg_data.set_value(i, 'Title', bg_data['Year Published'][i])
        bg_data.set_value(i, 'Year Published', 0)
bg_data['Year Published'] = pd.to_numeric(bg_data['Year Published'], errors='ignore')
bg_data['Year Published'] = bg_data['Year Published'].astype(int)
bg_data['Min Playing Time'] = bg_data['Play Time'].apply(lambda x: x.split(',')[0].strip('[]'))
bg_data['Max Playing Time'] = bg_data['Play Time'].apply(lambda x: x.split(',')[1].strip('[]'))
bg_data = bg_data.drop('Play Time', axis=1)
bg_data['Min Playing Time'] = pd.to_numeric(bg_data['Min Playing Time'], errors='coerce')
bg_data['Max Playing Time'] = pd.to_numeric(bg_data['Max Playing Time'], errors='coerce')
bg_data['Min Playing Time'] = bg_data['Min Playing Time'].fillna(0)
bg_data['Max Playing Time'] = bg_data['Max Playing Time'].fillna(0)
bg_data['Min Playing Time'] = bg_data['Min Playing Time'].astype(int)
bg_data['Max Playing Time'] = bg_data['Max Playing Time'].astype(int)
bg_data['Age'] = pd.to_numeric(bg_data['Age'], errors='coerce')
bg_data['Age'] = bg_data['Age'].fillna(bg_data['Age'].mean())
bg_data['Age'] = bg_data['Age'].astype(int)
bg_data['Complexity'] = bg_data['Complexity'].astype(float)
bg_data['Min Players'] = pd.to_numeric(bg_data['Min Players'], errors='coerce')
bg_data['Min Players'] = bg_data['Min Players'].fillna(bg_data['Min Players'].mean())
bg_data['Min Players'] = bg_data['Min Players'].astype(int)
bg_data['Max Players'] = pd.to_numeric(bg_data['Max Players'], errors='coerce')
bg_data['Max Players'] = bg_data['Max Players'].fillna(bg_data['Max Players'].mean())
bg_data['Max Players'] = bg_data['Max Players'].astype(int)
bg_data['Min Best Players'] = bg_data['Best Players'].apply(lambda x: x.split('\xe2\x80\x93')[0].strip('+'))
bg_data['Min Best Players'] = pd.to_numeric(bg_data['Min Best Players'], errors='coerce')
bg_data['Min Best Players'] = bg_data['Min Best Players'].fillna(bg_data['Min Best Players'].mean())
bg_data['Min Best Players'] = bg_data['Min Best Players'].astype(int)
bg_data['Max Best Players'] = bg_data['Best Players']
for i in xrange(len(bg_data)):
    bg_data.set_value(i, 'Designers', bg_data.loc[i, 'Designers'].strip('[]').split(', '))
    bg_data.set_value(i, 'Artists', bg_data.loc[i, 'Artists'].strip('[]').split(', '))
    bg_data.set_value(i, 'Publishers', bg_data.loc[i, 'Publishers'].strip('[]').split(', '))
    bg_data.set_value(i, 'Categories', bg_data.loc[i, 'Categories'].strip('[]').split(', '))
    bg_data.set_value(i, 'Mechanisms', bg_data.loc[i, 'Mechanisms'].strip('[]').split(', '))
    try:
        bg_data.set_value(i, 'Max Best Players', bg_data['Best Players'][i].split('\xe2\x80\x93')[1].strip('+'))
    except IndexError:
        bg_data.set_value(i, 'Max Best Players', bg_data['Best Players'][i].split('\xe2\x80\x93')[0].strip('+'))
bg_data['Max Best Players'] = pd.to_numeric(bg_data['Max Best Players'], errors='coerce')
bg_data['Max Best Players'] = bg_data['Max Best Players'].fillna(bg_data['Max Best Players'].mean())
bg_data['Max Best Players'] = bg_data['Max Best Players'].astype(int)
bg_data = bg_data.drop('Best Players', axis=1)

In [215]:
bg_data.loc[ind]

,Title,Year Published,Min Players,Max Players,Age,Complexity,Designers,Artists,Publishers,Categories,Mechanisms,Min Playing Time,Max Playing Time,Min Best Players,Max Best Players
1054,Carrom,0,2,4,6,1.51,[(Uncredited)],[],"[(Public Domain), (Unknown), ABRA, ASS Altenbu...",[Action / Dexterity],[Partnerships],60,60,2,2
1779,Unpublished Prototype,0,2,5,10,2.40,[(Uncredited)],[(Uncredited)],"[(Unknown), (Unpublished)]",[],[],0,0,1,1
2194,Outside the Scope of BGG,0,2,5,10,1.66,[(Uncredited)],[(Uncredited)],[(Unknown)],[Expansion for Base-game],[],0,0,3,3
2316,Traditional Card Games,0,2,5,10,2.05,[(Uncredited)],[(Uncredited)],[(Public Domain)],"[Card Game, Game System]",[],0,0,3,3
2648,Dobble: Kostenlose Demoversion,0,2,8,6,1.03,[Denis Blanchot],[],"[Asmodee, Bergsala Enigma, Gém Klub Kft., Kais...","[Children's Game, Party Game, Real-time]",[Pattern Recognition],5,5,4,5
2771,Celebrities,0,4,30,10,1.21,[(Uncredited)],[],[(Public Domain)],"[Humor, Party Game]","[Acting, Memory, Paper-and-Pencil, Partnerships]",90,90,6,6
3050,Riichi Mahjong,0,4,4,8,2.73,[(Uncredited)],[],[(Public Domain)],[Abstract Strategy],"[Betting/Wagering, Hand Management, Set Collec...",120,120,4,4
3260,Miscellaneous Game Accessory,0,2,5,10,3.47,[(Uncredited)],[(Uncredited)],[(Unknown)],[],[],0,0,1,1
3772,Koi-Koi,0,2,4,10,1.67,[(Uncredited)],[],"[(Public Domain), Nintendo Co., Ltd.]",[Card Game],"[Hand Management, Set Collection]",15,30,2,2
3991,KUBB,0,2,12,3,1.29,[],[],"[Bex Sport AB, Scanditoy]",[Action / Dexterity],[],15,15,3,3


In [247]:
bg_ranking_data = pd.read_csv('data/game_rankings.csv')

In [256]:
bg_ranking_data['Year Published'] = bg_ranking_data['Year Published'].fillna(0)
bg_ranking_data['Year Published'] = bg_ranking_data['Year Published'].astype(int)

In [257]:
bg_ranking_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13400 entries, 0 to 13399
Data columns (total 6 columns):
Rank              13400 non-null int64
Title             13400 non-null object
Year Published    13400 non-null int64
Geek Rating       13400 non-null float64
Avg Rating        13400 non-null float64
Num Ratings       13400 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 628.2+ KB


In [258]:
full_dataset_merge_new = pd.merge(bg_data, bg_ranking_data, how='outer', on=['Title', 'Year Published'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)

In [261]:
full_dataset_merge_new['Rank'] = full_dataset_merge_new['Rank'].astype(int)

ValueError: Cannot convert NA to integer

In [235]:
from collections import Counter

In [259]:
rank_count = Counter(full_dataset_merge_new['Rank'])

In [260]:
rank_count

Counter({nan: 1,
         nan: 1,
         2.0: 1,
         3.0: 1,
         4.0: 1,
         5.0: 1,
         nan: 1,
         7.0: 1,
         8.0: 1,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         12.0: 1,
         13.0: 1,
         14.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         23.0: 1,
         24.0: 1,
         25.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         29.0: 1,
         30.0: 1,
         nan: 1,
         32.0: 1,
         33.0: 1,
         34.0: 1,
         35.0: 1,
         36.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         40.0: 1,
         41.0: 1,
         42.0: 1,
         43.0: 1,
         44.0: 1,
         45.0: 1,
         46.0: 1,
         47.0: 1,
         48.0: 1,
         49.0: 1,
         50.0: 1,
         51.0: 1,
         52.0: 1,
         53.0: 1,
         54.0: 1,
         55.0: 1,
   

In [252]:
full_dataset_merge_new.set_index('Rank').sort_index()

,Title,Year Published,Min Players,Max Players,Age,Complexity,Designers,Artists,Publishers,Categories,Mechanisms,Min Playing Time,Max Playing Time,Min Best Players,Max Best Players,Geek Rating,Avg Rating,Num Ratings
Rank,,,,,,,,,,,,,,,,,,
1.0,Pandemic Legacy: Season 1,2015.0,2.0,4.0,13.0,2.81,"[Rob Daviau, Matt Leacock]",[Chris Quilliams],"[Z-Man Games, Asterion Press, Devir, Filosofia...","[Environmental, Medical]","[Action Point Allowance System, Co-operative P...",60.0,60.0,4.0,4.0,8.469,8.66,15951.0
2.0,Through the Ages: A New Story of Civilization,2015.0,2.0,4.0,14.0,4.29,[Vlaada Chvátil],"[Jakub Politzer, Milan Vavroň]","[Cranio Creations, Czech Games Edition, Devir,...","[Card Game, Civilization, Economic]","[Action Point Allowance System, Auction/Biddin...",180.0,240.0,3.0,3.0,8.306,8.77,6279.0
3.0,Twilight Struggle,2005.0,2.0,2.0,13.0,3.53,"[Ananda Gupta, Jason Matthews]","[Viktor Csete, Rodger B. MacGowan, Chechu Niet...","[GMT Games, (Self-Published), Asterion Press, ...","[Modern Warfare, Political, Wargame]","[Area Control / Area Influence, Campaign / Bat...",180.0,180.0,2.0,2.0,8.226,8.36,26424.0
4.0,Terra Mystica,2012.0,2.0,5.0,12.0,3.93,"[Jens Drögemüller, Helge Ostertag]",[Dennis Lohausen],"[Feuerland Spiele, Bard Centrum Gier, Cranio C...","[Civilization, Economic, Fantasy, Territory Bu...","[Route/Network Building, Variable Phase Order,...",60.0,150.0,4.0,4.0,8.153,8.29,22791.0
5.0,Star Wars: Rebellion,2016.0,2.0,4.0,14.0,3.56,[Corey Konieczka],[],"[ADC Blackfire Entertainment, Asterion Press, ...","[Movies / TV / Radio theme, Science Fiction, W...","[Area Control / Area Influence, Area Movement,...",180.0,240.0,2.0,2.0,8.130,8.56,7286.0
6.0,Scythe,2016.0,1.0,5.0,14.0,3.32,[Jamey Stegmaier],[Jakub Rozalski],"[Stonemaier Games, Albi, Feuerland Spiele, Fir...","[Civilization, Economic, Exploration, Fighting...","[Area Control / Area Influence, Grid Movement,...",90.0,115.0,4.0,4.0,8.082,8.35,12711.0
7.0,7 Wonders Duel,2015.0,2.0,2.0,10.0,2.25,"[Antoine Bauza, Bruno Cathala]",[Miguel Coimbra],"[Repos Production, ADC Blackfire Entertainment...","[Ancient, Card Game, City Building, Civilization]","[Card Drafting, Set Collection]",30.0,30.0,2.0,2.0,8.029,8.20,18324.0
8.0,Caverna: The Cave Farmers,2013.0,1.0,7.0,12.0,3.79,[Uwe Rosenberg],[Klemens Franz],"[Lookout Games, 999 Games, Crowd Games, Devir,...","[Animals, Economic, Fantasy, Farming]","[Tile Placement, Worker Placement]",30.0,210.0,4.0,4.0,8.022,8.19,16163.0
9.0,The Castles of Burgundy,2011.0,2.0,4.0,12.0,3.04,[Stefan Feld],"[Julien Delval, Harald Lieske]","[alea, Ravensburger Spieleverlag GmbH, Broadwa...","[Dice, Medieval, Territory Building]","[Dice Rolling, Set Collection, Simultaneous Ac...",30.0,90.0,2.0,2.0,7.994,8.11,24014.0


In [253]:
full_dataset_merge_new[full_dataset_merge_new['Title'] == 'Cosmic Encounter']

,Title,Year Published,Min Players,Max Players,Age,Complexity,Designers,Artists,Publishers,Categories,Mechanisms,Min Playing Time,Max Playing Time,Min Best Players,Max Best Players,Rank,Geek Rating,Avg Rating,Num Ratings
2728,Cosmic Encounter,1977.0,2.0,6.0,12.0,2.37,"[Bill Eberle, Jack Kittredge, Bill Norton, Pet...","[John Blanche, Chris Ferguson, Anders Jeppsson...","[Alga, ASS Altenburger Spielkarten, Descartes ...","[Bluffing, Negotiation, Science Fiction]","[Hand Management, Variable Player Powers]",90.0,90.0,4.0,5.0,698.0,6.597,6.93,3617.0
2729,Cosmic Encounter,1991.0,3.0,6.0,10.0,2.64,"[Bill Eberle, Jack Kittredge, Bill Norton, Pet...",[Gerald O'Malley],"[Games Workshop Ltd., Hexagames (I), Mayfair G...","[Bluffing, Card Game, Science Fiction]",[Variable Player Powers],60.0,60.0,6.0,6.0,1157.0,6.308,7.22,848.0
2730,Cosmic Encounter,2000.0,2.0,4.0,10.0,2.32,"[Bill Eberle, Jack Kittredge, Bill Norton, Pet...",[Dimitri Patelis],[Avalon Hill (Hasbro)],"[Bluffing, Card Game, Science Fiction]",[Variable Player Powers],60.0,60.0,4.0,4.0,2046.0,5.976,6.41,1095.0
2731,Cosmic Encounter,2008.0,3.0,5.0,12.0,2.55,"[Bill Eberle, Jack Kittredge, Bill Norton, Pet...","[Andrew Navaro, Ryan Barger, Felicia Cano, Sab...","[Fantasy Flight Games, Arclight, Asterion Pres...","[Bluffing, Negotiation, Science Fiction, Space...","[Hand Management, Partnerships, Take That, Var...",60.0,120.0,5.0,5.0,87.0,7.463,7.61,18398.0


Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            13855, 13856, 13857, 13858, 13859, 13860, 13861, 13862, 13863,
            13864],
           dtype='int64', length=13865)